In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading
Loading EPA's Smart Location Database <br>
Loading survey data from Baltimore Ecosystem Study <br>
Merging the two data sources on Census Block Group (GEOID10). This removes many of the rows from the SLD.

In [2]:
sl_data = gpd.read_file('data/SmartLocationDatabaseV3/SmartLocationDatabase.gdb')

In [3]:
survey_data = pd.read_csv('data/survey/BESTS_1999_2000_2003_2006_2011_ANON.csv', low_memory=False)
# GEOID_2010 seems to be corrupted (have missing information) for some cities
survey_data = survey_data.drop(labels='GEOID_2010', axis=1)
survey_data.GISJOIN_2010 = survey_data.GISJOIN_2010.astype('str')
survey_data.GISJOIN_2010 = survey_data.GISJOIN_2010.apply(lambda x: x[1:3]+x[4:7]+x[8:])
# Rename GISJOIN_2010 to GEOID10 to match SLD
survey_data = survey_data.rename({'GISJOIN_2010':'GEOID10'}, axis=1)

In [4]:
df = pd.merge(survey_data, sl_data, on='GEOID10')
print('Number of samples:',len(df))

Number of samples: 14298


## Data Preprocessing

Adding in covariates for analysis

In [5]:
# Sex: Female is 1, Male is 0
df['sex_code'] = pd.Series(np.zeros(len(df)))
df.loc[df.sex=='male','sex_code'] = 0
df.loc[df.sex=='female','sex_code'] = 1

# Race: different column for each race category in the survey data
df['black_code'] = pd.Series(np.zeros(len(df)))
df.loc[df.race=='black','black_code'] = 1
df['hispanic_code'] = pd.Series(np.zeros(len(df)))
df.loc[df.race=='hispanic','hispanic_code'] = 1
df['white_code'] = pd.Series(np.zeros(len(df)))
df.loc[df.race=='white','white_code'] = 1
df['asian_code'] = pd.Series(np.zeros(len(df)))
df.loc[df.race=='asian','asian_code'] = 1

# Age and Education level: recode as floats
df.age = df.age.replace('dk refused', np.nan)
df.age = df.age.astype('float')
df.edu = df.edu.replace('dk refused', np.nan)
df.edu = df.edu.astype('float')

In [6]:
# Recode income categories to integers (increasing with increased income)
def income_to_value(row):
    val = np.NaN
    if row.income2 == 'refused': val = 0
    if row.income2 == 'under $15K': val = 0
    if row.income2 == '$15K to $25K': val = 1
    if row.income2 == '$25K to $35K': val = 2
    if row.income2 == '$35K to $50K': val = 3
    if row.income2 == '$50K to $75K': val = 4
    if row.income2 == '$75K to $100K': val = 5
    if row.income2 == 'over $100K': val = 6
    return val
df['Income_Value'] = df.apply(income_to_value, axis=1)

In [7]:
# Add city string based on the state number in the SLD
# Note: this workaround only works because each analyzed city is in a different state
df['State_Number'] = df.GEOID10.apply(lambda x: x[:2])
def state_to_city(row):
    city = np.NaN
    if row.State_Number == '25' or row.State_Number == '33': city = 'Boston' # Boston
    if row.State_Number == '24': city = 'Baltimore' # Baltimore
    if row.State_Number == '27': city = 'Twin_Cities' # Twin Cities
    if row.State_Number == '12': city = 'Orlando' # Orlando
    if row.State_Number == '04': city = 'Phoenix' # Phoenix
    if row.State_Number == '06': city = 'LA' # LA
    return city
df['City'] = df.apply(state_to_city, axis=1)
city_hot = pd.get_dummies(df['City'])
df = df.join(city_hot)

## Finalize dataset for SEM analysis and save to file
SEM analysis completed in `R`

In [15]:
sem_data = pd.DataFrame()
# Cohesion data
sem_data[['Close_Knit','Trust','Known_Neighs','Will_Help']] = df[['closeKnit','trust','knownNeighs','willHelp']]
# Diversity of built environment data
sem_data[['Emp_HH_Entropy','Emp_Entropy']] = df[['D2A_EPHHM','D2B_E8MIXA']] # both pretty normal
# Physical density data
sem_data[['Intersection_Density','Path_Density']] = df[['D3B','D3APO']]
# Social density data
sem_data[['HH_Density','Pop_Density','Emp_Density']] = df[['D1A','D1B','D1C']]
# Neighborhood connectedness data
sem_data[['Transit_Dist','Transit_Service']] = df[['D4A','D4D']]
# Covariates
sem_data[['Age','Sex','Education','Income','White','Black','Hispanic','Asian','City']] = df[['age','sex_code','edu','Income_Value','white_code','black_code','hispanic_code','asian_code','City']]

# Necessary Transformations
sem_data.Transit_Dist[sem_data.Transit_Dist==-99999] = 1500
sem_data.Transit_Service[sem_data.Transit_Service==-99999] = 0
sem_data['Transit_Proximity'] = -sem_data['Transit_Dist']

print('Number of observations with NAs:',len(sem_data))
sem_data = sem_data.dropna()
print('Number of observations without NAs:',len(sem_data))

Number of observations with NAs: 14298
Number of observations without NAs: 9670


/Users/andrew/opt/miniconda3/envs/walkability/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andrew/opt/miniconda3/envs/walkability/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
sem_data.to_csv('sem_data.csv')